pyquation is set up to minimize the effort required to use equations, not necisarily for defining them. It shouldn't be too hard, especially once you have a bunch of Variables set up. At some point, I aim to include a bunch of pre-defined Variables and Equations to the package.

In [2]:
from pyquation import Equation, Variable
# For dimensions, quantities, and constants
from sympy.physics.units import *
# For prefixes like milli and mega
from sympy.physics.units.prefixes import *
# For a more complete (but not symbolic) list of units and constants, see
# https://docs.scipy.org/doc/scipy/reference/constants.html
# from scipy.constants import *

### Dimensions
First, specify any previously unspecified dimensions. sympy already has a lot specified, but if you would like more, this is how you would do it

In [3]:
height = Dimension('height', 'h')

### Quantities
Sympy has a lot of units already specified, but in case you need more, this is how you make new ones. 
For more help, see the sympy docs:
https://docs.sympy.org/latest/modules/physics/units/index.html

In [4]:
stories = Quantity("stories", abbrev="st")
stories.set_global_dimension(height)

### Variables
These are all the variables in the equations we'll be using. For each of them, you need to specify
the dimension, and the default units it's in. These will be the units in your equations
(for example, newtons = kilogram * meters/second^2, but newtons != pounds * meters/second^2)
If you don't provide a symbol, it will automatically use the one specified by the Dimension you
passed it. If there's not one, or you want to change it, you can specify a symbol or symbols.

In [5]:
F = Variable(force, newtons, description='May it be with you.')
M = Variable(mass, kilogram)
A = Variable(acceleration, meters/second**2, ('A', 'accel'))
v = Variable(velocity, meters/second, 'v')
# We want the full name to be "radius", not "length"
r = Variable(length, meters, 'r', name='radius')

### Equations
This is how you define the equations themselves. Equation(`left hand side`, `right hand side`, `optional description`).

In [6]:
newtons_law = Equation(F, M*A, "This is Newton's 2nd law of motion")
centripetal_motion = Equation(A, (v**2)/r)

In [7]:
# If you ever forget how to use the equation, or what the variables are, you can use the help() method
newtons_law.help()

Eq(F, A*M)

where:
	M is mass in units of kilogram
	F is force in units of newton
		May it be with you.
	A is acceleration in units of meter/second**2

This is Newton's 2nd law of motion


* You call equations just like function, by either their full name or their symbol. 
* Don't forget to include units!
* You can also specify show_solved_term=True to see the term it automatically solved for
* Units will automatically be converted to the default unit of the term we're solving for (i.e. the units used in the equation)

In [8]:
assert newtons_law(M=3*kg, F=12*newtons) == newtons_law(mass=3*kg, force=12*newtons)
newtons_law(M=3*pounds, F=12*newtons, show_solved_term=True)

Eq(A, 400000000*meter/(45359237*second**2))

If you don't want the answer in the default units, you can instead tell it what units you would like it in. If set to None, or something that doesn't make sense, it will give the default units. 

In [9]:
newtons_law(F=3*newtons, A=2*meters/second**2, show_solved_term=True, units=pounds)

Eq(M, 150000000*pound/45359237)

If you don't have enough variables known, and you want to solve symbolically for the other variables, you can do that too!

In [10]:
newtons_law(M=3*kg)

Eq(F, 3*kilogram*A)

Equations can also handle uncertainty. Uncertainties can be specified by either `<name>_unc`, `unc_<name>`, or `δ<name>`. Uncertainties should be plain numbers, or sympy expressions, and shouldn't include units.

In [11]:
accel, unc = newtons_law(M=3*kg, F=12*newtons, M_unc=1, unc_force=3)
display(accel)
display(unc)

4*meter/second**2

5/3

Uncertainties only work if all but 1 of the variables are specified (otherwise it doesn't really make
much sense: how do you get the uncertainty of an equality?)
However, if you only specify one uncertainty, it will still calculate the uncertainty symbolically

In [12]:
newtons_law(M=3*kg, F=12*newtons, δF=1)[1]

sqrt(16*δM**2 + 1)/3

### Combining Equations
You can combine multiple Equations the normal way:

In [13]:
accel = newtons_law(M=3*pounds, F=12*newtons)
centripetal_motion(A=accel, v=10*meters/second, show_solved_term=True)

Eq(r, 45359237*meter/4000000)

You can also combine multiple equations by adding them together, then calling them the same way you call a single Equation (though it does have a slightly different interface). The new equation stuffs everything together and calculates everything it can from what you give it

In [14]:
radius_of_a_centrifuge = newtons_law + centripetal_motion
radius_of_a_centrifuge(M=3*kg, F=12*newton, v=10*meters/second)

{M: 3*kilogram,
 F: 12*newton,
 v: 10*meter/second,
 A: 4*meter/second**2,
 r: 25*meter}

This works the same way with uncertainties as well

In [17]:
radius_of_a_centrifuge(M=3*kg, F=12*newton, v=10*meters/second, unc_M=1, unc_force=2, v_unc=3)

{M: 3*kilogram,
 F: 12*newton,
 v: 10*meter/second,
 A: 4*meter/second**2,
 r: 25*meter,
 δM: 1,
 δF: 2,
 δv: 3,
 δA: 2*sqrt(5)/3,
 δr: 5*sqrt(449)/6}

Note that adding equations together works slightly differently (it instaniates a `CombinedEquation` class), so if you want to combine multiple equations "properly" in a certain way, simply making a new, larger Equation might be a better idea.